In [1]:
import pandas as pd
import numpy as np
import gensim
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
stopWords = stopwords.words('english')
from nltk import word_tokenize
from scipy.spatial.distance import cosine, cityblock, euclidean, minkowski
from nltk.metrics import jaccard_distance
from nltk.metrics import edit_distance
import xgboost
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.metrics import make_scorer,precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score 
import matplotlib.pyplot as plt


In [2]:
print(stopWords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Inspiration for implementation 

https://towardsdatascience.com/string-matching-with-fuzzywuzzy-e982c61f8a84

https://www.analyticsvidhya.com/blog/2021/07/fuzzy-string-matching-a-hands-on-guide/

https://markroxor.github.io/gensim/static/notebooks/WMD_tutorial.html

https://towardsdatascience.com/identifying-duplicate-questions-on-quora-top-12-on-kaggle-4c1cf93f1c30

https://stackoverflow.com/questions/22129943/how-to-calculate-the-sentence-similarity-using-word2vec-model-of-gensim-with-pyt

In [ ]:

df = pd.read_csv('Data/archive/questions.csv')
df = df.dropna(how="any").reset_index(drop=True)
mcClass = max(df.is_duplicate.value_counts())
print("baseline accuracy: ", mcClass / len(df))

In [ ]:
df.drop(['id', 'qid1', 'qid2'], axis=1, inplace=True)

In [ ]:
# make sure questions are strings
df['question1'] = df['question1'].apply(str)
df['question2'] = df['question2'].apply(str)
df['question1'] = df['question1'].str.strip()
df['question2'] = df['question2'].str.strip()


In [ ]:

# apply basic feature engineering

df['Q1Len'] = df.question1.apply(lambda row: len(row))
df['Q2Len'] = df.question2.apply(lambda row: len(row))
df['LenDiff'] = abs(df.Q1Len - df.Q2Len)
df['charCountQ1'] = df.question1.apply(lambda row: len(row.replace(" ", "")))
df['charCountQ2'] = df.question2.apply(lambda row: len(row.replace(" ", "")))
df['wordCountQ1'] = df.question1.apply(lambda row: len(row.split()))
df['wordCountQ2'] = df.question2.apply(lambda row: len(row.split()))
df['charCountDifference']=df.apply(lambda row: abs(row['charCountQ1']- row['charCountQ2']),axis=1)
df['WordCountDifference']=df.apply(lambda row: abs(row['wordCountQ1'] - row['wordCountQ2']),axis=1)
df['sharedWords'] = df.apply(lambda row: len(set(row['question1'].lower().split()).intersection(set(row['question2'].lower().split()))), axis=1)
df['fuzzyRatio'] = df.apply(lambda row: fuzz.ratio(row['question1'], row['question2']), axis=1)
df['fuzzyPartialRatio'] = df.apply(lambda row: fuzz.partial_ratio(row['question1'], row['question2']), axis=1)
df['fuzzyPartialTokenSetRatio'] = df.apply(lambda row: fuzz.partial_token_set_ratio(row['question1'], row['question2']), axis=1)
df['fuzzyPartialSokenSortRatio'] = df.apply(lambda row: fuzz.partial_token_sort_ratio(row['question1'], row['question2']), axis=1)
df['fuzzyTokenSetRatio'] = df.apply(lambda row: fuzz.token_set_ratio(row['question1'], row['question2']), axis=1)
df['fuzzyTokenSortRatio'] = df.apply(lambda row: fuzz.token_sort_ratio(row['question1'], row['question2']), axis=1)
df['LastWordSame']=df.apply(lambda row: int(row['question1'].split()[-1] == row['question2'].split()[-1]),axis=1)
df['FirstWordSame']=df.apply(lambda row: int(row['question1'].split()[0] == row['question2'].split()[0]),axis=1)


In [ ]:
# get part of words that are shared between the questions, jaccard and edit distance
def wordSharePart(row):
    question1 = set(row['question1'].lower().split())
    question2 = set(row['question2'].lower().split())  

    return len(question1 & question2)/(len(question1) + len(question2))   
df['sharedWordsPart'] = df.apply(wordSharePart, axis=1)

def jaccardDistance(row):
    question1 = set(row['question1'].lower().split())
    question2 = set(row['question2'].lower().split()) 
    return jaccard_distance(question1, question2)  
df['jaccardDistance'] = df.apply(jaccardDistance, axis=1)

def editDistance(row):
    return edit_distance(row['question1'], row['question2'], substitution_cost=1)
df['editDistance'] = df.apply(editDistance, axis=1)

In [ ]:
# get word movers distance for words in the questions that are not in the NLTK stopwords

def wmdW2v(question1, question2):
    question1 = question1.lower().split()
    question2 = question2.lower().split()
    question1 = [word for word in question1 if word not in stopWords]
    question2 = [word for word in question2 if word not in stopWords]
    return w2vModel.wmdistance(question1, question2)

def wmdW2vNormed(question1, question2):
    question1 = question1.lower().split()
    question2 = question2.lower().split()
    question1 = [word for word in question1 if word not in stopWords]
    question2 = [word for word in question2 if word not in stopWords]
    return w2vModelNormed.wmdistance(question1, question2)

w2vModel = gensim.models.KeyedVectors.load_word2vec_format('Data/w2v/GoogleNews-vectors-negative300.bin.gz', binary=True)
df['wmdW2v'] = df.apply(lambda row: wmdW2v(row['question1'], row['question2']), axis=1)

w2vModelNormed = gensim.models.KeyedVectors.load_word2vec_format('Data/w2v/GoogleNews-vectors-negative300.bin.gz', binary=True)
w2vModelNormed.init_sims(replace=True)
df['wmdW2vNormed'] = df.apply(lambda row: wmdW2vNormed(row['question1'], row['question2']), axis=1)

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec

def wmdGlove(question1, question2):
    question1 = question1.lower().split()
    question2 = question2.lower().split()
    question1 = [word for word in question1 if word not in stopWords]
    question2 = [word for word in question2 if word not in stopWords]
    return gloveModel.wmdistance(question1, question2)

def wmdGloveNormed(question1, question2):
    question1 = question1.lower().split()
    question2 = question2.lower().split()
    question1 = [word for word in question1 if word not in stopWords]
    question2 = [word for word in question2 if word not in stopWords]
    return gloveModelNormed.wmdistance(question1, question2)

glove_filename = 'Data/glove/glove.6B.300d.txt'
word2vec_output_file = glove_filename+'.word2vec'
glove2word2vec(glove_filename, word2vec_output_file)
word2vec_output_file = glove_filename+'.word2vec'
gloveModel = gensim.models.KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
gloveModelNormed = gensim.models.KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
gloveModelNormed.init_sims(replace=True) 

df['wmdGlove'] = df.apply(lambda row: wmdGlove(row['question1'], row['question2']), axis=1)
df['wmdGloveNormed'] = df.apply(lambda row: wmdGloveNormed(row['question1'], row['question2']), axis=1)

In [ ]:
# w2v & glove representations of the questions


def getw2vEmbedding(s):
    words = word_tokenize(s.lower())
    words = [word for word in words if not word in stopWords]
    words = [word for word in words if word.isalpha()]
    vecs = []
    for w in words:
        try:
            vecs.append(w2vModel[w])
        except:
            continue
    vecs = np.array(vecs)
    vecs = vecs.sum(axis=0)
    vecs = vecs / np.linalg.norm(vecs)
    sum = np.sum(vecs)
    if np.isnan(sum):
        return np.random.randn(300)
    else:
        return vecs / np.linalg.norm(vecs)


def getGloveEmbedding(s):
    words = word_tokenize(s.lower())
    words = [word for word in words if not word in stopWords]
    words = [word for word in words if word.isalpha()]
    vecs = []
    for w in words:
        try:
            vecs.append(gloveModel[w])
        except:
            continue
    vecs = np.array(vecs)
    vecs = vecs.sum(axis=0)
    vecs = vecs / np.linalg.norm(vecs)
    sum = np.sum(vecs)
    if np.isnan(sum):
        return np.random.randn(300)
    else:
        return vecs / np.linalg.norm(vecs)



In [ ]:
question1W2vVecs = np.zeros((df.shape[0], 300))

for i, question in enumerate(df.question1.values):
    question1W2vVecs[i, :] = getw2vEmbedding(question)
    
question2W2vVecs  = np.zeros((df.shape[0], 300))
for i, question in enumerate(df.question2.values):
    question2W2vVecs[i, :] = getw2vEmbedding(question)

question1GloveVecs = np.zeros((df.shape[0], 300))

for i, question in enumerate(df.question1.values):
    question1GloveVecs[i, :] = getGloveEmbedding(question)
    
question2GloveVecs  = np.zeros((df.shape[0], 300))
for i, question in enumerate(df.question2.values):
    question2GloveVecs[i, :] = getGloveEmbedding(question)

df['cosineDistW2v'] = [cosine(x, y) for (x, y) in zip(question1W2vVecs, question2W2vVecs)]
df['cityblockDistW2v'] = [cityblock(x, y) for (x, y) in zip(question1W2vVecs, question2W2vVecs)]
df['euclideanDistW2v'] = [euclidean(x, y) for (x, y) in zip(question1W2vVecs, question2W2vVecs)]
df['minkowskiDistW2v'] = [minkowski(x, y) for (x, y) in zip(question1W2vVecs, question2W2vVecs)]
df['cosineDistGlove'] = [cosine(x, y) for (x, y) in zip(question1GloveVecs, question2GloveVecs)]
df['cityblockDistGlove'] = [cityblock(x, y) for (x, y) in zip(question1GloveVecs, question2GloveVecs)]
df['euclideanDistGlove'] = [euclidean(x, y) for (x, y) in zip(question1GloveVecs, question2GloveVecs)]
df['minkowskiDistGlove'] = [minkowski(x, y) for (x, y) in zip(question1GloveVecs, question2GloveVecs)]

In [ ]:
df.drop(['question1', 'question2'], axis=1, inplace=True)


In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(how='all')

In [ ]:
df.columns.tolist()

In [ ]:

X = df.loc[:, df.columns != 'is_duplicate']
y = df.loc[:, df.columns == 'is_duplicate']
Y = y.values.ravel()

# stratified 10-fold cross validation evaluation of xgboost model

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1' : make_scorer(f1_score)}


xgbModel = xgboost.XGBClassifier()
kfold = StratifiedKFold(n_splits=10)
results = cross_validate(xgbModel, X, Y, cv=kfold,scoring=scoring)



In [ ]:
print(results)
print('acc: ', np.mean(results['test_accuracy']), np.std(results['test_accuracy']),'prec: ',np.mean(results['test_precision']),np.std(results['test_precision']),'recall: ',np.mean(results['test_recall']),np.std(results['test_recall']),'f1: ',np.mean(results['test_f1']),np.std(results['test_f1']))

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
xgbModel = xgboost.XGBClassifier().fit(X_train, y_train.values.ravel()) 
pred = xgbModel.predict(X_test)
confusionMatrix = confusion_matrix(y_test, pred)  

In [ ]:
print('Test CM: ', confusionMatrix)  
print('Accuracy: ', accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
xgboost.plot_importance(xgbModel)
plt.figure(figsize = (30, 50))
plt.show()